In [1]:
a = set([1,2,3,4])

In [4]:
b = set([2,3,4, 7,8])

In [5]:
a & b

{2, 3, 4}

In [40]:
import pandas as pd
import random
import numpy as np

data = pd.read_csv('data/view_log.csv')

In [45]:
user_item = data.groupby('userID')['articleID'].apply(list).reset_index()
item_pool = data['articleID'].unique()
print(user_item)

         userID                                          articleID
0     USER_0000  [ARTICLE_0661, ARTICLE_2316, ARTICLE_1345, ART...
1     USER_0001  [ARTICLE_1408, ARTICLE_0202, ARTICLE_2406, ART...
2     USER_0002  [ARTICLE_2504, ARTICLE_2803, ARTICLE_1910, ART...
3     USER_0003  [ARTICLE_2115, ARTICLE_1092, ARTICLE_0958, ART...
4     USER_0004  [ARTICLE_0173, ARTICLE_2259, ARTICLE_0690, ART...
...         ...                                                ...
1410  USER_1416  [ARTICLE_0399, ARTICLE_2703, ARTICLE_0399, ART...
1411  USER_1417  [ARTICLE_0340, ARTICLE_2335, ARTICLE_1992, ART...
1412  USER_1418  [ARTICLE_1446, ARTICLE_0693, ARTICLE_2416, ART...
1413  USER_1419  [ARTICLE_2364, ARTICLE_2223, ARTICLE_1743, ART...
1414  USER_1420  [ARTICLE_2779, ARTICLE_0614, ARTICLE_2122, ART...

[1415 rows x 2 columns]


In [25]:
user_item['isTest'] = user_item['articleID'].apply(lambda x: True if len(x) >= 6 and random.random() < 0.2  else False)
user_item

,userID,articleID,isTest
0,USER_0000,"[ARTICLE_0661, ARTICLE_2316, ARTICLE_1345, ART...",True
1,USER_0001,"[ARTICLE_1408, ARTICLE_0202, ARTICLE_2406, ART...",False
2,USER_0002,"[ARTICLE_2504, ARTICLE_2803, ARTICLE_1910, ART...",False
3,USER_0003,"[ARTICLE_2115, ARTICLE_1092, ARTICLE_0958, ART...",False
4,USER_0004,"[ARTICLE_0173, ARTICLE_2259, ARTICLE_0690, ART...",False
...,...,...,...
1410,USER_1416,"[ARTICLE_0399, ARTICLE_2703, ARTICLE_0399, ART...",False
1411,USER_1417,"[ARTICLE_0340, ARTICLE_2335, ARTICLE_1992, ART...",False
1412,USER_1418,"[ARTICLE_1446, ARTICLE_0693, ARTICLE_2416, ART...",False
1413,USER_1419,"[ARTICLE_2364, ARTICLE_2223, ARTICLE_1743, ART...",True


In [34]:
test_data = user_item[user_item['isTest']].drop(columns=['isTest', 'userID']).reset_index(drop=True)
train_data = user_item[~user_item['isTest']].drop(columns=['isTest', 'userID']).reset_index(drop=True)

In [92]:
test_data

,articleID
0,"[ARTICLE_0661, ARTICLE_2316, ARTICLE_1345, ART..."
1,"[ARTICLE_1513, ARTICLE_2651, ARTICLE_0553, ART..."
2,"[ARTICLE_2972, ARTICLE_0107, ARTICLE_0336, ART..."
3,"[ARTICLE_2719, ARTICLE_0212, ARTICLE_1541, ART..."
4,"[ARTICLE_2245, ARTICLE_2651, ARTICLE_0929, ART..."
...,...
194,"[ARTICLE_1947, ARTICLE_1745, ARTICLE_1015, ART..."
195,"[ARTICLE_0998, ARTICLE_0588, ARTICLE_1947, ART..."
196,"[ARTICLE_1884, ARTICLE_1325, ARTICLE_0809, ART..."
197,"[ARTICLE_2894, ARTICLE_1601, ARTICLE_2372, ART..."


In [96]:
test_data['y'] = test_data['articleID'].apply(lambda x: x[-5:])
test_data.iloc[0].y

['ARTICLE_1230',
 'ARTICLE_2992',
 'ARTICLE_2255',
 'ARTICLE_2806',
 'ARTICLE_0411']

In [67]:
import copy
import math

def add_noise(all_items: np.ndarray, user_items:list, noise_level:float=0.2, ):
    user_item_copy = copy.deepcopy(user_items)
    num_noisy_item = math.ceil(len(user_item_copy) * noise_level)
    for _ in range(num_noisy_item):
        item = random.choice(all_items)
        while item in user_items:
            item = random.choice(all_items)
            
        user_item_copy.pop(0)
        insert_index = random.randint(0, len(user_item_copy))
        user_item_copy.insert(insert_index, item)
    return user_item_copy
            

In [72]:
def add_subset(
    user_items: list,
    min_size: int,
):
    size = random.randint(min_size, len(user_items))
    start_pos = random.randint(0, len(user_items) - size)
    return user_items[start_pos: start_pos + size]

In [100]:
data

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [104]:
data.index

RangeIndex(start=0, stop=42717, step=1)

In [106]:
def transfrom_and_split_test_and_train_data(data: pd.DataFrame):
    user_item = data.groupby("userID")["articleID"].apply(list).reset_index()
    item_pool = data["articleID"].unique()
    user_item["isTest"] = user_item["articleID"].apply(
        lambda x: True if len(x) >= 6 and random.random() < 0.2 else False
    )
    test_data = (
        user_item[user_item["isTest"]]
        .drop(columns=["isTest", "userID"])
        .reset_index(drop=True)
    )

    train_data = (
        user_item[~user_item["isTest"]]
        .drop(columns=["isTest", "userID"])
        .reset_index(drop=True)
    )

    return train_data, test_data, item_pool

In [128]:
import os
import pickle

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

def load_label_encoder(pkl_path, data):
    label_encoder = LabelEncoder()
    if not os.path.exists(pkl_path):
        label_encoder.fit(data)

        with open(pkl_path, "wb") as f:
            pickle.dump(label_encoder, f)

    else:
        with open(pkl_path, "rb") as f:
            label_encoder = pickle.load(f)

    return label_encoder

In [130]:
article_encoder = LabelEncoder()
article_encoder.fit(data['articleID'])

LabelEncoder()

In [132]:
article_encoder.transform(data['articleID'])

array([ 632, 2212, 1282, ..., 1764,   29, 2314])

In [134]:
data['articleID']=article_encoder.transform(data['articleID'])

In [116]:
def split_y_from_data(data: pd.DataFrame, label):
    data["y"] = data[label].str[-5:]
    data[label] = data[label].str[:-5]
    return data


In [135]:
tr, te, ip = transfrom_and_split_test_and_train_data(data=data)

In [136]:
tr

,articleID
0,"[2394, 2685, 1825, 2685, 1825, 2685, 2685, 959..."
1,"[2019, 1049, 918, 1777]"
2,"[2479, 1210, 2160, 1507, 539, 891, 2125, 1507,..."
3,"[917, 1056, 2632, 1550, 561, 1043, 917, 297, 2..."
4,"[1445, 2537, 528, 840, 1114, 668, 140, 840, 20..."
...,...
1212,"[379, 2589, 379, 2589, 2589, 2589, 379]"
1213,"[323, 2230, 1903, 1840, 2725, 1904, 1081, 1903..."
1214,"[1382, 662, 2307, 2186, 662, 2307, 2307, 1176,..."
1215,"[2258, 2125, 1665, 1665, 2666, 171, 2666, 2666..."


In [137]:
te

,articleID
0,"[632, 2212, 1282, 1046, 1417, 991, 991, 2156, ..."
1,"[1344, 191, 2298, 1467, 2383, 1160, 2746, 2743..."
2,"[163, 2160, 659, 1239, 2802, 1816, 872, 1860, ..."
3,"[2844, 102, 319, 2876, 1445, 1741, 2702, 812, ..."
4,"[480, 928, 2473, 183, 536, 902, 1472, 209, 250..."
...,...
193,"[1129, 1944, 518, 345, 258, 2002, 517, 410, 25..."
194,"[2759, 2841, 1447, 1369, 152, 2291, 1453, 65, ..."
195,"[2609, 125, 2297, 2758, 2321, 662, 2286, 2233,..."
196,"[2537, 2576, 1578, 300, 1425, 1190, 891, 858, ..."


In [138]:
ip

array([ 632, 2212, 1282, ...,  683, 1634, 1654])

In [139]:
te_ = split_y_from_data(data=te, label='articleID')

In [140]:
from scipy.sparse import csr_matrix

def create_interaction_matrix(data: pd.DataFrame, item_col: str, all_items: np.ndarray):
    num_of_users = len(data)
    num_of_items = len(all_items)

    rows = data.index.repeat(data[item_col].str.len()).values
    cols = np.concatenate(data[item_col].values)
    interaction = np.ones(len(cols), dtype=int)

    interaction_matrix = csr_matrix(
        (interaction, (rows, cols)), shape=(num_of_users, num_of_items)
    )
    return interaction_matrix

In [142]:
tri=create_interaction_matrix(data=tr, item_col='articleID', all_items=ip)

In [145]:
tri.todense()[0]

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [147]:
a = [1,2,3,4]
def tesss(a:list):
    # a = copy.deepcopy(a)
    a.pop(0)
    return a

b= tesss(a)
print(a)
print(b)

[2, 3, 4]
[2, 3, 4]


In [149]:
tr

,articleID
0,"[2394, 2685, 1825, 2685, 1825, 2685, 2685, 959..."
1,"[2019, 1049, 918, 1777]"
2,"[2479, 1210, 2160, 1507, 539, 891, 2125, 1507,..."
3,"[917, 1056, 2632, 1550, 561, 1043, 917, 297, 2..."
4,"[1445, 2537, 528, 840, 1114, 668, 140, 840, 20..."
...,...
1212,"[379, 2589, 379, 2589, 2589, 2589, 379]"
1213,"[323, 2230, 1903, 1840, 2725, 1904, 1081, 1903..."
1214,"[1382, 662, 2307, 2186, 662, 2307, 2307, 1176,..."
1215,"[2258, 2125, 1665, 1665, 2666, 171, 2666, 2666..."


In [152]:
tra = tr['articleID']

In [154]:
for item_list in tra:
    print(item_list)

[2394, 2685, 1825, 2685, 1825, 2685, 2685, 959, 1500, 1542, 2685, 2685, 1542, 2666, 959, 2685, 2685, 2685, 2685, 959, 2685, 1964, 2685, 2685, 959]
[2019, 1049, 918, 1777]
[2479, 1210, 2160, 1507, 539, 891, 2125, 1507, 2874, 986, 1371, 172, 1477, 1294, 932, 2125, 1507, 2125]
[917, 1056, 2632, 1550, 561, 1043, 917, 297, 2632, 1408, 1455, 1640, 218, 721, 1640, 2281, 1550, 2779, 2632, 1016, 971, 725, 23, 1550, 439]
[1445, 2537, 528, 840, 1114, 668, 140, 840, 2069, 939, 2505, 668, 253, 637]
[379, 1712, 379, 1665, 1597, 1550, 662, 1375, 1183, 1550, 1665, 2346, 2482, 379, 1713, 379, 1923, 1210, 2431, 2011, 1550, 620, 1923, 1550, 1923, 2223, 2298, 1837, 1550, 743]
[1517, 2746, 565]
[698, 1878, 2666, 2666, 977, 2466, 2795]
[2605, 201, 1472, 1472, 1318, 480, 473, 1935, 2005, 2005]
[9, 209, 1454, 2723, 2723, 1573, 2160, 675, 675, 9]
[1375, 1303, 1805, 1972, 166, 2673, 325, 759, 401, 1922, 2673, 2760, 2673, 350, 1805, 34, 1394, 1303, 347, 1937, 369, 2275, 369, 2746, 34, 1993, 2673, 2673, 296, 2187

In [155]:
a = 1
b = np.int64(1)

a == b

np.True_

In [157]:
import torch

In [160]:
user= torch.tensor([ 331,  608,  821,  886,  987,  621,  348,  279, 1191,  755,  533,  357,
         871,  826, 1055,  926,  201,   62,  944, 1249, 1109,  308, 1413, 1356,
        1013,   15, 1288,  463,  385,  694, 1379, 1073,  607,  924,  351,   11,
           1,  939, 1053,  353, 1039, 1259, 1201,  925, 1140,  477,  969,  901,
        1363, 1038,  134,   33,   14,  961,  740,  768, 1216,  962,  638,  767,
         647,  972,  407,  560,  195,  514,  743,  570,   32,  932, 1304,  681,
        1234,  184, 1014,  688, 1361,  178,  389,  341,  632,  876,  453,   27,
         171,  537,  314,  541,   22, 1381, 1399,  643,  787,  415,  841, 1084,
         268,   66,  454,  515,  622,   83, 1274, 1015,  746,  161,  731,   85,
        1130, 1268,  243,  133,  188,  796,  750,  806,  440,  943,  462,  815,
        1079,  594,  289,  863,   55,  445, 1046, 1283])

user

tensor([ 331,  608,  821,  886,  987,  621,  348,  279, 1191,  755,  533,  357,
         871,  826, 1055,  926,  201,   62,  944, 1249, 1109,  308, 1413, 1356,
        1013,   15, 1288,  463,  385,  694, 1379, 1073,  607,  924,  351,   11,
           1,  939, 1053,  353, 1039, 1259, 1201,  925, 1140,  477,  969,  901,
        1363, 1038,  134,   33,   14,  961,  740,  768, 1216,  962,  638,  767,
         647,  972,  407,  560,  195,  514,  743,  570,   32,  932, 1304,  681,
        1234,  184, 1014,  688, 1361,  178,  389,  341,  632,  876,  453,   27,
         171,  537,  314,  541,   22, 1381, 1399,  643,  787,  415,  841, 1084,
         268,   66,  454,  515,  622,   83, 1274, 1015,  746,  161,  731,   85,
        1130, 1268,  243,  133,  188,  796,  750,  806,  440,  943,  462,  815,
        1079,  594,  289,  863,   55,  445, 1046, 1283])

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
a=  nn.Parameter(
    torch.Tensor(1, 10),
    requires_grad=False
)

In [17]:
type(optim.Adam())

type

In [12]:
a = torch.Tensor([1,2,3,4])
b= a.mul(3)
b

tensor([ 3.,  6.,  9., 12.])

In [1]:
a = [1,2,3,4]

In [3]:
mean(a)

NameError: name 'mean' is not defined